In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

In [2]:
total_pop = pd.read_csv('../data_eda/total_pop.csv')
man_pop = pd.read_csv('../data_eda/man_pop.csv')
woman_pop = pd.read_csv('../data_eda/woman_pop.csv')
woman_rep = pd.read_csv('../data_eda/woman_rep.csv')
children_pop = pd.read_csv('../data_eda/children_pop.csv')
youth_pop = pd.read_csv('../data_eda/youth_pop.csv')
elder_pop = pd.read_csv('../data_eda/elder_pop.csv')

In [3]:
#pd.concat([total_pop.set_index('geo_location'),man_pop.set_index('geo_location')], axis = 1)
# total_pop.merge(man_pop, how = 'outer', left_on= 'geo_location', right_on = 'geo_location')
# 7264499 rows

# Combine dataset

In [4]:
def change_column_name(df,string):
    for column in df.columns:
        df.rename(columns = {column : str(column + string)}, inplace = True)
    return df.columns

change_column_name(total_pop,'_total')
change_column_name(man_pop, '_man')
change_column_name(woman_pop,'_woman')
change_column_name(woman_rep, '_woman_rep')
change_column_name(children_pop,'_children')
change_column_name(youth_pop,'_youth')
change_column_name(elder_pop,'_elder')

Index(['latitude_elder', 'longitude_elder', 'population_elder',
       'geo_location_elder'],
      dtype='object')

In [5]:
t0 = time.time()
full_df = pd.concat([
        total_pop.set_index('geo_location_total'),
        man_pop.set_index('geo_location_man'),
        woman_pop.set_index('geo_location_woman'),
        woman_rep.set_index('geo_location_woman_rep'),
        children_pop.set_index('geo_location_children'),
        youth_pop.set_index('geo_location_youth'),
        elder_pop.set_index('geo_location_elder')
        ], 
        axis = 1)
print (time.time() - t0)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


56.60473990440369


In [6]:
full_df.shape

(7264499, 21)

# create df with only population with one complete lat, lon

In [7]:
pop_df = full_df[[col for col in full_df.columns if col.startswith('population')]]

In [8]:
pop_df.head(10)

,population_total,population_man,population_woman,population_woman_rep,population_children,population_youth,population_elder
"21.9995833,36.7259722",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"21.9995833,36.7265278",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"21.9998611,36.7259722",5.85038,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"21.9998611,36.7265278",5.85038,NaN,NaN,NaN,NaN,NaN,NaN
"22.0001389,36.7259722",5.85038,NaN,NaN,NaN,NaN,NaN,NaN
"22.0001389,36.72625",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"22.0001389,36.810416700000005",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"22.0004167,36.724861100000005",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"22.0004167,36.7251389",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629
"22.0004167,36.725694399999995",NaN,3.143666,2.673635,1.460095,0.638118,1.140843,0.204629


In [9]:
# reset index
pop_df.reset_index(inplace = True)

In [10]:
pop_df.rename(columns = {'index' : 'coordinate'}, inplace = True)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [11]:
pop_df.dtypes

coordinate               object
population_total        float64
population_man          float64
population_woman        float64
population_woman_rep    float64
population_children     float64
population_youth        float64
population_elder        float64
dtype: object

In [12]:
# split coordinate into latitude, longtitude
latitude = []
longitude = []
for i in pop_df['coordinate']:
    lat = i.split(',')[0]
    lon = i.split(',')[1]
    latitude.append(lat)
    longitude.append(lon)

In [13]:
# add latitude, longitude
pop_df['latitude'] = latitude
pop_df['longitude'] = longitude

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
pop_df.columns

Index(['coordinate', 'population_total', 'population_man', 'population_woman',
       'population_woman_rep', 'population_children', 'population_youth',
       'population_elder', 'latitude', 'longitude'],
      dtype='object')

In [15]:
# drop coordinate
pop_df.drop(columns = 'coordinate', inplace = True)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
pop_df.isnull().sum()

population_total        2610572
population_man          2610572
population_woman        2610572
population_woman_rep    2625483
population_children     2625483
population_youth        2610572
population_elder        2610572
latitude                      0
longitude                     0
dtype: int64

In [17]:
pop_df.dtypes

population_total        float64
population_man          float64
population_woman        float64
population_woman_rep    float64
population_children     float64
population_youth        float64
population_elder        float64
latitude                 object
longitude                object
dtype: object

In [18]:
pop_df['latitude'] = pop_df['latitude'].str.strip().astype(float)
pop_df['longitude'] = pop_df['longitude'].str.strip().astype(float)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Cairo Latitude: 30.033333, Longitude: 31.233334

In [19]:
df_cairo = pop_df.loc[(pop_df['Latitude'] > 29.8) & 
                  (pop_df['Latitude'] < 30.2) & 
                  (pop_df['Lognitude'] > 31.1) & 
                  (pop_df['Lognitude'] < 31.4)]

KeyError: 'Latitude'

In [ ]:
df_cairo.to_csv('../data_eda/df_cairo.csv', index = False)